In [1]:
from pathlib import Path
import pygame
from stable_baselines3 import PPO
from gymnasium.wrappers import TimeLimit
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.callbacks import EvalCallback, CallbackList
from stable_baselines3.common.vec_env import VecNormalize
from stable_baselines3.common.env_util import make_vec_env
from torch import nn
from deform_rl.algos.save_manager import get_paths, consistency_check, delete_experiment, load_manager
from deform_rl.algos.training.training_helpers import *
from deform_rl.envs.Rectangle_env.debug_env import *
from deform_rl.algos.lego.networks import *

pygame 2.6.1 (SDL 2.28.4, Python 3.10.12)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [3]:
from stable_baselines3 import PPO



# Create environment


# Wrap in DummyVecEnv
vec_env = make_vec_env(lambda: TimeLimit(RectVel(),1000), n_envs=4)

# Add VecNormalize
vec_env = VecNormalize(vec_env, norm_obs=True, norm_reward=False)


# Train the agent
model = PPO("MlpPolicy", vec_env, verbose=1,device='cpu',tensorboard_log="./ppo_rectvel_tensorboard/")
model.learn(total_timesteps=500000,tb_log_name="ppo_rectvel")

# Save model and normalization stats
model.save("ppo_vecnormalize")
vec_env.save("vec_normalize.pkl")

# Load model and normalization stats
# vec_env = VecNormalize.load("vec_normalize.pkl", make_vec_env(lambda: RectVel(), n_envs=1))
# model = PPO.load("ppo_vecnormalize", vec_env,device='cpu')

# Evaluate
# vec_env.training = False
# vec_env.norm_reward = False
# obs = vec_env.reset()
# done = False
# while not done:
#     action, _states = model.predict(obs)
#     obs, reward, done, info = vec_env.step(action)
#     # print(reward)


Using cpu device
Logging to ./ppo_rectvel_tensorboard/ppo_rectvel_17


/home/michal/Documents/Skola/bakalarka/RL/RL-cable/.venv/lib/python3.10/site-packages/gymnasium/spaces/box.py:235: UserWarning: WARN: Box low's precision lowered by casting to float32, current low.dtype=float64
  gym.logger.warn(
/home/michal/Documents/Skola/bakalarka/RL/RL-cable/.venv/lib/python3.10/site-packages/gymnasium/spaces/box.py:305: UserWarning: WARN: Box high's precision lowered by casting to float32, current high.dtype=float64
  gym.logger.warn(


---------------------------------
| rollout/           |          |
|    ep_len_mean     | 1e+03    |
|    ep_rew_mean     | 4.51e+03 |
| time/              |          |
|    fps             | 4843     |
|    iterations      | 1        |
|    time_elapsed    | 1        |
|    total_timesteps | 8192     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 957         |
|    ep_rew_mean          | 3.75e+03    |
| time/                   |             |
|    fps                  | 2837        |
|    iterations           | 2           |
|    time_elapsed         | 5           |
|    total_timesteps      | 16384       |
| train/                  |             |
|    approx_kl            | 0.006717425 |
|    clip_fraction        | 0.0586      |
|    clip_range           | 0.2         |
|    entropy_loss         | -2.82       |
|    explained_variance   | -0.00318    |
|    learning_rate        | 0.

KeyboardInterrupt: 

In [4]:
# Load model and normalization stats
# vec_env = make_vec_env(lambda: TimeLimit(TrajectoryVel(render_mode='human'),1000), n_envs=1,seed=25)
vec_env = VecNormalize.load("vec_normalize.pkl", make_vec_env(lambda: TimeLimit(TrajectoryVel(render_mode='human'),1000), n_envs=1))
model = PPO.load("ppo_vecnormalize", vec_env,device='cpu')
# vec_env = make_vec_env(lambda: TrajectoryVel(render_mode='human'), n_envs=1,seed=25)
# Evaluate
vec_env.training = False
vec_env.norm_reward = False
obs = vec_env.reset()
run=True
cum = 0
for i in range(10):
        done = False
        cur = 0
        obs = vec_env.reset()
        while not done:
                action, _states = model.predict(obs,deterministic=True)
                # print(np.linalg.norm(action))
                obs, reward, done, info = vec_env.step(action)
                # obs, reward, done, info = vec_env.step(obs[:2]/np.linalg.norm(obs[:2]))
                vec_env.render()
                # print(reward)
                cum += reward
                cur += reward
                if pygame.event.get(pygame.QUIT):
                        run=False
                        break
                # print(reward)
        if not run:
                break
        print(cur)
vec_env.close()
print(cum)

[1572.7002]
[1658.1616]
[1673.5332]
[1343.2904]
[1471.709]
[1197.4427]
[1433.7554]
[1374.3655]
[1079.4602]
[1291.6525]
[14096.069]


In [9]:
env.save("./shit_norms.pkl")
model.save("./shit_model")

In [3]:
t_env = make_vec_env(lambda: RectVel(render_mode='human'), n_envs=1)
t_env = VecNormalize.load("./shit_norms.pkl",t_env)
model = PPO.load("./shit_model",t_env,device='cpu')

for i in range(5):
    done = False
    obs = t_env.reset()
    while not done:
        action, _ = model.predict(obs, deterministic=True)
        obs, reward, done, info = t_env.step(action)
        t_env.render()
        if pygame.event.get(pygame.QUIT):
            break
t_env.close()



/home/michal/Documents/Skola/bakalarka/RL/RL-cable/.venv/lib/python3.10/site-packages/gymnasium/spaces/box.py:235: UserWarning: WARN: Box low's precision lowered by casting to float32, current low.dtype=float64
  gym.logger.warn(
/home/michal/Documents/Skola/bakalarka/RL/RL-cable/.venv/lib/python3.10/site-packages/gymnasium/spaces/box.py:305: UserWarning: WARN: Box high's precision lowered by casting to float32, current high.dtype=float64
  gym.logger.warn(
